# Imports

In [1]:
import aurorax
import datetime
import pprint
import os
import pandas as pd

# Upload ephemeris data

In [2]:
# read API key from environment vars
api_key = os.environ["AURORAX_API_KEY"]

In [3]:
# set values
program = "notebook-program"
platform = "notebook-platform"
instrument_type = "notebook-instrument-type"
data_product_type = "keogram"
url = "https://data.aurorax.space/test-url.jpg"
metadata = {
    "test_meta1": "testing1",
    "test_meta2": "testing2",
}
start_dt = datetime.datetime(2020, 1, 1, 0, 0)
end_dt = datetime.datetime(2020, 1, 1, 0, 59)

In [4]:
# get the ephemeris source ID
source = aurorax.sources.get_using_filters(program=[program],
                                           platform=[platform],
                                           instrument_type=[instrument_type])
identifier = source["data"][0]["identifier"]
print("Identifier: %d" % (identifier))

Identifier: 356


In [5]:
# create DataProduct object
e = aurorax.data_products.DataProduct(identifier,
                                      program,
                                      platform,
                                      instrument_type,
                                      start_dt,
                                      end_dt,
                                      url,
                                      data_product_type,
                                      metadata)
pprint.pprint(e)

{'data_product_type': 'keogram',
 'end_dt': datetime.datetime(2020, 1, 1, 0, 59),
 'identifier': 356,
 'instrument_type': 'notebook-instrument-type',
 'metadata': {'test_meta1': 'testing1', 'test_meta2': 'testing2'},
 'platform': 'notebook-platform',
 'program': 'notebook-program',
 'start_dt': datetime.datetime(2020, 1, 1, 0, 0),
 'url': 'https://data.aurorax.space/test-url.jpg'}


In [6]:
# set records array
#
# we do this because you can upload multiple ephemeris records, so the 
# standard object being passed to the upload function is a list
records = []
records.append(e)

In [7]:
# upload record
res = aurorax.data_products.upload(api_key, identifier, records=records)

In [8]:
# check the response to see if the upload succeeded
if (res["status_code"] == 202):
    print("Successfully uploaded")
else:
    print("Failed to upload")
    pprint.pprint(res)

Successfully uploaded


# Check that the data was uploaded by retrieving it

In [11]:
# set values
programs = [program]
platforms = [platform]
instrument_types = [instrument_type]
data_product_types = [data_product_type]

# perform search
s = aurorax.data_products.search(start_dt,
                                 end_dt,
                                 programs=programs,
                                 platforms=platforms,
                                 instrument_types=instrument_types,
                                 data_product_types=data_product_types,
                                 show_progress=True)

[2020-11-23 00:51:56.009296] Search object created
[2020-11-23 00:51:58.387732] Request submitted
[2020-11-23 00:51:58.388609] Request ID: 80e7d5d1-dbbd-4be2-bf6d-9add2a37d550
[2020-11-23 00:51:58.389086] Request details available at: http://api.staging.aurorax.space/api/v1/data_products/requests/80e7d5d1-dbbd-4be2-bf6d-9add2a37d550
[2020-11-23 00:51:58.439803] Checking for data ...
[2020-11-23 00:52:00.805610] Request has data available, retrieving it ...
[2020-11-23 00:52:00.849635] Retrieved 5 Bytes of data containing 0 records, completed search request


In [ ]:
# show data as a pandas dataframe
df = pd.DataFrame(s["data"])
df